In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as npr
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from keras.optimizers import Adam
from keras_nlp.layers import PositionEmbedding

In [3]:
seed = 428

np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

In [4]:
def get_masked_input_and_labels(encoded_texts, n_cat):
    # For each sentence, mask each word one-by-one

    encoded_texts_masked = []
    y_labels = []

    for encoded_text in encoded_texts:
        for i in range(len(encoded_text)):
            encoded_text_masked = np.copy(encoded_text)
            y_label = encoded_text_masked[i]
            encoded_texts_masked.append(np.delete(encoded_text_masked, i))
            y_labels.append(np.array([y_label]))

    return np.array(encoded_texts_masked), np.array(y_labels)

In [5]:
#### K = number of countries = number of capitals = number of currencies
#### S = number of words used by both topics
#### L = sentence length
#### q1, q2 = probability of having 1 or 2 pairs
#### embed_dim = dimension of embeddings
#### n_sentences = number of training sentences

def train_model(K, S, L, q1, q2, embed_dim, n_sentences):
    
    countries = ['country_' + str(i) for i in range(K)]
    capitals = ['capital_' + str(i) for i in range(K)]
    currencies = ['currency_' + str(i) for i in range(K)]
    randoms = ['random_' + str(i) for i in range(S)]

    vocabs = countries + capitals + currencies + randoms
    vocab_map = {}

    for i in range(len(vocabs)):
        vocab_map[vocabs[i]] = i
        
    sentences = []
    sentences_number = []
    
    q0 = 1 - q1 - q2

    for i in range(n_sentences):

        sentence = []
        
        temp = npr.uniform()
        temp2 = npr.uniform()
        
        if temp2 <= q0:
            n_pairs = 0
        elif temp2 <= q0 + q1:
            n_pairs = 1
        else:
            n_pairs = 2
        
        if temp <= 0.5: ### country - capital
        
            pairs = np.random.choice(np.arange(K), n_pairs, replace = False)
            for pair in pairs:
                sentence.append(countries[pair])
                sentence.append(capitals[pair])        
                 
        else: ### country - currency
            
            pairs = np.random.choice(np.arange(K), n_pairs, replace = False)
            for pair in pairs:
                sentence.append(countries[pair])
                sentence.append(currencies[pair])        
            
        sentence += list(np.random.choice(randoms, L - 2 * n_pairs, replace = False))  

        sentence_number = [vocab_map[i] for i in sentence]
        sentences.append(sentence)
        sentences_number.append(sentence_number)
        
    x_train = np.array(sentences_number)
    n_cat = len(vocab_map)
    x_masked_train, y_masked_labels_train = get_masked_input_and_labels(x_train, n_cat)
    
    callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
    inputs = layers.Input((x_masked_train.shape[1],), dtype=tf.int64)
    word_embeddings = layers.Embedding(n_cat, embed_dim, name="word_embedding")(inputs)
    encoder_output = layers.GlobalAveragePooling1D()(word_embeddings)
    mlm_output = layers.Dense(n_cat, name="mlm_cls", activation="softmax", use_bias=False)(encoder_output)
    mlm_model = keras.Model(inputs = inputs, outputs = mlm_output)
    adam = Adam()
    mlm_model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)

    history = mlm_model.fit(x_masked_train, y_masked_labels_train,
                        validation_split = 0.5, callbacks = [callback], 
                        epochs=500, batch_size=128, verbose=0)
    
    return sentences, vocab_map, mlm_model

In [12]:
def get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples):
    
    sentences, vocab_map, current_model = train_model(K, S, L, q1, q2, embed_dim, n_sentences)

    acc_capitals = []
    prob_capitals = []

    for _ in range(n_samples):
        sentence = []
        random_capitals = np.random.choice(np.arange(K), int(L/2), replace = False)
        for random_capital in random_capitals:
            sentence.append('country_' + str(random_capital))
            sentence.append('capital_' + str(random_capital))
        sentence = sentence[:-1]
        sentence_number = [vocab_map[i] for i in sentence]
        temp = keras.backend.function(inputs = current_model.layers[0].input, outputs = current_model.layers[-1].output) \
            (np.array(sentence_number).reshape(1,len(sentence_number)))
        actual = vocab_map['capital_' + str(random_capitals[-1])]
        acc_capitals.append(1 if np.argsort(-1 * temp)[0][0] == actual else 0)
        prob_capitals.append(temp[0][vocab_map['capital_' + str(random_capitals[-1])]])
        
    acc_currencies = []
    prob_currencies = []

    for _ in range(n_samples):
        sentence = []
        random_currencies = np.random.choice(np.arange(K), int(L/2), replace = False)
        for random_currency in random_currencies:
            sentence.append('country_' + str(random_currency))
            sentence.append('currency_' + str(random_currency))
        sentence = sentence[:-1]
        sentence_number = [vocab_map[i] for i in sentence]
        temp = keras.backend.function(inputs = current_model.layers[0].input, outputs = current_model.layers[-1].output) \
            (np.array(sentence_number).reshape(1,len(sentence_number)))
        actual = vocab_map['currency_' + str(random_currencies[-1])]
        acc_currencies.append(1 if np.argsort(-1 * temp)[0][0] == actual else 0)
        prob_currencies.append(temp[0][vocab_map['currency_' + str(random_currencies[-1])]])
        

    return sentences, current_model, vocab_map, (np.mean(acc_capitals), np.mean(prob_capitals)), \
                (np.mean(acc_currencies), np.mean(prob_currencies))

In [13]:
K = 10 # number of countries
L = 8 # sentence length
S = 20 # number of words used by both topics
embed_dim = 10 # CBOW embedding dimension
n_sentences = 50000 # number of sentences in the training set
n_samples = 1000

In [14]:
q0 = 0 # probability of having 0 pairs
q1 = 1 # probability of having 1 pair
q2 = 0 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(0.0, 6.7673636e-33)
(0.0, 5.6979238e-33)
(0.0, 2.1064791e-27)
(0.0, 1.0648849e-27)
(0.0, 1.173342e-28)
(0.0, 1.3984568e-27)
(0.0, 3.4738882e-32)
(0.0, 8.263466e-32)
(0.0, 3.3604521e-28)
(0.0, 3.1755494e-29)
(0.0, 2.5013111e-27)
(0.0, 4.5719487e-28)
(0.0, 1.1592604e-27)
(0.0, 6.189103e-27)
(0.0, 1.1733562e-28)
(0.0, 1.036676e-27)
(0.0, 2.5578754e-29)
(0.0, 4.9096465e-29)
(0.0, 1.9697492e-31)
(0.0, 1.1850897e-28)
(0.0, 6.36358292382107e-28)
(0.0, 1.0345764839254664e-27)


In [15]:
q0 = 0 # probability of having 0 pairs
q1 = 0 # probability of having 1 pair
q2 = 1 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(0.0, 1.3654233e-17)
(0.0, 5.8669104e-18)
(0.0, 5.2788557e-18)
(0.0, 2.156302e-17)
(0.0, 1.1036926e-16)
(0.0, 2.8656812e-15)
(0.0, 3.204862e-19)
(0.0, 1.14058925e-20)
(0.0, 1.247297e-15)
(0.0, 6.411275e-13)
(0.0, 2.4965474e-28)
(0.0, 1.0113966e-20)
(0.0, 2.4611956e-13)
(0.0, 1.6268546e-11)
(0.0, 1.3519926e-16)
(0.0, 2.3635387e-18)
(0.0, 4.3553932e-20)
(0.0, 3.704575e-17)
(0.0, 3.2156094e-11)
(0.0, 1.0231353e-13)
(0.0, 3.240372548069571e-12)
(0.0, 1.7014919115106223e-12)


In [16]:
q0 = 1/2 # probability of having 0 pairs
q1 = 1/2 # probability of having 1 pair
q2 = 0 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(0.529, 0.50681126)
(0.483, 0.49430284)
(0.473, 0.50388)
(0.551, 0.49443257)
(0.59, 0.5162015)
(0.425, 0.48489732)
(0.455, 0.49721435)
(0.541, 0.50345355)
(0.644, 0.51130277)
(0.409, 0.4924527)
(0.499, 0.5050634)
(0.524, 0.4981152)
(0.809, 0.5217616)
(0.174, 0.47519344)
(0.418, 0.4933404)
(0.583, 0.5065843)
(0.504, 0.5020764)
(0.474, 0.49612716)
(0.426, 0.48922327)
(0.522, 0.50795805)
(0.5347000000000001, 0.5046874970197678)
(0.4686, 0.4953517138957978)


In [17]:
q0 = 1/2 # probability of having 0 pairs
q1 = 0 # probability of having 1 pair
q2 = 1/2 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(1.0, 1.0)
(1.0, 0.9999986)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 0.9999818)
(1.0, 1.0)
(1.0, 0.9999998)
(1.0, 0.9999995)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 0.9999995)
(1.0, 0.99998367)
(1.0, 0.9999995)
(1.0, 0.99999404)
(1.0, 0.99999684)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 0.99996775)
(1.0, 1.0)
(0.9999999999999999, 0.9999927103519439)
(0.9999999999999999, 0.9999994039535524)


In [20]:
q0 = 0 # probability of having 0 pairs
q1 = 1/2 # probability of having 1 pair
q2 = 1/2 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(1.0, 0.99883693)
(1.0, 0.99841136)
(1.0, 0.9997456)
(1.0, 0.999078)
(1.0, 0.99979854)
(1.0, 0.9995016)
(1.0, 0.99952036)
(1.0, 0.9993807)
(1.0, 0.9978059)
(1.0, 0.9980908)
(1.0, 0.9993821)
(1.0, 0.9980849)
(1.0, 0.99912375)
(1.0, 0.99919546)
(1.0, 0.9999166)
(1.0, 0.9993853)
(1.0, 0.9996006)
(1.0, 0.9990403)
(1.0, 0.9990687)
(1.0, 0.99917686)
(0.9999999999999999, 0.9992799043655395)
(0.9999999999999999, 0.9989345252513886)


In [19]:
q0 = 1/3 # probability of having 0 pairs
q1 = 1/3 # probability of having 1 pair
q2 = 1/3 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(1.0, 0.9998982)
(1.0, 0.9999511)
(1.0, 0.99937797)
(1.0, 0.9996614)
(1.0, 0.99997777)
(1.0, 0.99956477)
(1.0, 0.99982727)
(1.0, 0.999904)
(1.0, 0.99995136)
(1.0, 0.9998977)
(1.0, 0.999957)
(1.0, 0.9999748)
(1.0, 0.9914531)
(1.0, 0.99930006)
(1.0, 0.9995866)
(1.0, 0.99959075)
(1.0, 0.99994814)
(1.0, 0.99986273)
(1.0, 0.999743)
(1.0, 0.9997454)
(0.9999999999999999, 0.9989720404148101)
(0.9999999999999999, 0.999745273590088)
